In [29]:
import os
from typing import Type
from langchain.chat_models import ChatOpenAI
from langchain.tools import BaseTool
from pydantic import BaseModel, Field
from langchain.agents import initialize_agent, AgentType
from langchain.tools import WikipediaQueryRun, DuckDuckGoSearchRun
from langchain.retrievers import WikipediaRetriever

# "gpt-3.5-turbo" "gpt-4o-mini"
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.1)


class SearchToolArgsSchema(BaseModel):
  query: str = Field(description="The query you will search for")


class WikipediaSearchTool(BaseTool):
  name: str = "WikipediaSearchTool"
  description: str = """
  Use this tool to find the information for a given topic.
  It takes a query as an argument.
  After using this tool, you must run DuckDuckGoSearchTool to get more information.
  Example query: OpenAI o3, Tesla Model3, Attempted assassination of Donald Trump
  """
  args_schema: Type[SearchToolArgsSchema]=SearchToolArgsSchema

  def _run(self, query):
    wiki_retriever = WikipediaRetriever(top_k_results=5)
    wiki = WikipediaQueryRun(api_wrapper=wiki_retriever)
    return wiki.run(query)


class DuckDuckGoSearchTool(BaseTool):
  name: str = "DuckDuckGoSearchTool"
  description: str = """
  Use this tool to find the information for a given topic.
  It takes a query as an argument.
  After running this tool, you must create a research file with the SaveResearchTool.
  Example query: OpenAI o3, Tesla Model3, Attempted assassination of Donald Trump

  """
  args_schema: Type[SearchToolArgsSchema] = SearchToolArgsSchema

  def _run(self, query):
    ddg = DuckDuckGoSearchRun()
    return ddg.run(query)


class SaveResearchArgsSchema(BaseModel):
  filename: str = Field(description="The name of the file to save the research results")
  content: str = Field(description="The content to save in the file")


class SaveResearchTool(BaseTool):
  name: str = "SaveResearchTool"
  description: str = """
  Use this tool to save research results to a text file.
  It takes a filename and content as arguments.
  The file will be saved in the ./research_results directory.
  """
  args_schema: Type[SaveResearchArgsSchema] = SaveResearchArgsSchema

  def _run(self, filename: str, content: str):
    # Create research_results directory if it doesn't exist
    os.makedirs("./research_results", exist_ok=True)

    # Ensure filename has .txt extension
    if not filename.endswith('.txt'):
      filename += '.txt'

    # Create full file path
    file_path = os.path.join("./research_results", filename)

    # Write content to file
    with open(file_path, 'w', encoding='utf-8') as f:
      f.write(content)

    return f"Research results saved to {file_path}"


agent = initialize_agent(
    llm=llm,
    verbose=True,
    agent=AgentType.OPENAI_FUNCTIONS,
    handle_parse_errors=True,
    tools=[
        WikipediaSearchTool(),
        DuckDuckGoSearchTool(),
        SaveResearchTool(),
    ],
)

prompt = "Research about the XZ backdoor"
#prompt = "Research about Trump Tariff Policy"

agent.invoke(prompt)



> Entering new AgentExecutor chain...

Invoking: `WikipediaSearchTool` with `{'query': 'XZ backdoor'}`




/opt/homebrew/anaconda3/envs/fullstackgpt/lib/python3.11/site-packages/wikipedia/wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file /opt/homebrew/anaconda3/envs/fullstackgpt/lib/python3.11/site-packages/wikipedia/wikipedia.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')


Page: XZ Utils backdoor
Summary: In February 2024, a malicious backdoor was introduced to the Linux build of the xz utility within the liblzma library in versions 5.6.0 and 5.6.1 by an account using the name "Jia Tan". The backdoor gives an attacker who possesses a specific Ed448 private key remote code execution through OpenSSH on the affected Linux system. The issue has been given the Common Vulnerabilities and Exposures number CVE-2024-3094 and has been assigned a CVSS score of 10.0, the highest possible score.
While xz is commonly present in most Linux distributions, at the time of discovery the backdoored version had not yet been widely deployed to production systems, but was present in development versions of major distributions. The backdoor was discovered by the software developer Andres Freund, who announced his findings on 29 March 2024.

Page: XZ Utils
Summary: XZ Utils (previously LZMA Utils) is a set of free software command-line lossless data compressors, including the pr

{'input': 'Research about the XZ backdoor',
 'output': 'I have researched the XZ backdoor, also known as CVE-2024-3094. Here are the key details:\n\n### XZ Backdoor (CVE-2024-3094)\n\nIn February 2024, a malicious backdoor was introduced to the Linux build of the xz utility within the liblzma library in versions 5.6.0 and 5.6.1 by an account using the name "Jia Tan". The backdoor allows an attacker who possesses a specific Ed448 private key to execute remote code through OpenSSH on the affected Linux system. This issue has been assigned the Common Vulnerabilities and Exposures (CVE) number CVE-2024-3094 and has been given a CVSS score of 10.0, the highest possible score.\n\nWhile xz is commonly present in most Linux distributions, at the time of discovery, the backdoored version had not yet been widely deployed to production systems but was present in development versions of major distributions. The backdoor was discovered by software developer Andres Freund, who announced his findings